In [4]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import PolynomialFeatures
from numpy import mean
from numpy import std
from tqdm import tqdm_notebook
import time
import datetime
import os
pd.options.display.precision = 15
from pathlib import Path
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
data_folder=Path("C:/Users/msi/Desktop/classification/project/project/")
file_to_open= data_folder/ "fout.csv"

In [23]:
%%time
df = pd.read_csv(file_to_open,names=['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'post', 'relationship', 'nation', 'gender', 'capital_gain', 'capital_loss', 'hours_per_week', 'country', 'wealth'],na_values=[' ?','?','? ',' ? '])


Wall time: 71.8 ms


In [24]:
print(df.isnull().sum())
print(df[['workclass','post']].isnull().all(axis='columns').sum())

age                  0
workclass         1375
fnlwgt               0
education            0
education_num        0
marital_status       0
post              1381
relationship         0
nation               0
gender               0
capital_gain         0
capital_loss         0
hours_per_week       0
country            455
wealth               0
dtype: int64
1375


In [25]:
df.drop(['fnlwgt', 'education', 'country'], axis=1, inplace=True)

In [26]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

le = LabelEncoder()
scaler = StandardScaler()
pl = PolynomialFeatures(2, include_bias=False)
df["wealth"] = le.fit_transform(df['wealth'])

In [27]:
columns_to_encoding = [ 'workclass', 'marital_status', 'post',
                        'relationship', 'nation', 'gender' ]

columns_to_normalize = [ 'age', 'education_num', 'hours_per_week', 
                         'capital_gain', 'capital_loss' ]
df = pd.get_dummies(df, columns=columns_to_encoding)

In [28]:
df.head()

,age,education_num,capital_gain,capital_loss,hours_per_week,wealth,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,nation_ Amer-Indian-Eskimo,nation_ Asian-Pac-Islander,nation_ Black,nation_ Other,nation_ White,gender_ F,gender_ M
0,50,13,0,0,13,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,28,13,0,0,40,0,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
2,37,14,0,0,40,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
3,52,9,0,0,45,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,31,14,14084,0,50,1,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0


In [29]:
features_ = df.drop(columns = ['wealth'])
target = df['wealth']
X_train, X_test, y_train, y_test = train_test_split(features_, target, test_size = 0.3,random_state = 0)

print ("Train data set size : ", X_train.shape)
print ("Test data set size : ", X_test.shape)

Train data set size :  (17160, 47)
Test data set size :  (7355, 47)


In [30]:
# for train set
X_temp = pl.fit_transform(X_train[columns_to_normalize])
X_temp = scaler.fit_transform(X_temp)
X_train.drop(columns_to_normalize, axis=1, inplace=True)
X_train = np.hstack((X_train.values, X_temp))

In [31]:
# for test set
X_temp = pl.transform(X_test[columns_to_normalize])
X_temp = scaler.transform(X_temp)
X_test.drop(columns_to_normalize, axis=1, inplace=True)
X_test = np.hstack((X_test.values, X_temp))

In [36]:
# !!! PCA!!!
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

param_distribution = {
    'n_estimators': [200, 400],
}

scoring = {    
    'Accuracy': make_scorer(accuracy_score),
    'F1_Score': make_scorer(fbeta_score, beta=1),    
}

result = []
#--------------------
for i in range(1, 13):
    # train
    pca = PCA(i)
    X_t = pca.fit_transform(X_train)
    search_cv = RandomizedSearchCV(RandomForestClassifier(), param_distribution,
                                   scoring=scoring, n_jobs=-1, 
                                   cv=StratifiedKFold(n_splits=10, shuffle=True), 
                                   refit='F1_Score') 
    search_cv.fit(X_t, y_train.values)
    model = search_cv.best_estimator_
    # test
    X_t = pca.transform(X_test)
    y_pred = model.predict(X_t)
    
    # model evaluation
    f1 = fbeta_score(y_test.values, y_pred, beta=1)
    acc = accuracy_score(y_test.values, y_pred)
    print(f"{i} {acc} {f1}")
    
    result.append((i, acc, f1, pca, model))

1 0.7443915703602991 0.4763231197771588
2 0.8076138681169273 0.56793893129771
3 0.8237933378653977 0.598014888337469
4 0.8296397008837525 0.6117136659436009
5 0.8256968048946295 0.6038318912237329
6 0.8361658735554045 0.6338498936493467
7 0.8426920462270564 0.6499243570347957
8 0.845819170632223 0.656155245603396
9 0.845819170632223 0.655528554070474
10 0.8450033990482665 0.6534954407294833
11 0.845819170632223 0.6565717746820109
12 0.8478585995921142 0.6603945371775418
